In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
df = pd.read_csv('SPAM text_msg.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


### count the number of ham and spam category

In [30]:
num_ham = 0 
num_spam =  0 
for i in range(len(df)):
    if df['Category'][i] == 'ham':
        num_ham = num_ham+1
    else:
        num_spam = num_spam+1
print(num_ham)
print(num_spam)


# ##########################
df.groupby('Category').describe()

4825
747


Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

#### Here we see tha the data is not in balance format so we need to balance it 


### Here we convert the category in to 0 , 1
### Ham = 0
### spam = 1

In [34]:
df['label'] = pd.get_dummies(df['Category'] , drop_first=True)
df.head()

,Category,Message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [35]:
X = df['Message']
y = df['label']

In [37]:
X[:5]   , y[:5]

(0    Go until jurong point, crazy.. Available only ...
 1                        Ok lar... Joking wif u oni...
 2    Free entry in 2 a wkly comp to win FA Cup fina...
 3    U dun say so early hor... U c already then say...
 4    Nah I don't think he goes to usf, he lives aro...
 Name: Message, dtype: object,
 0    0
 1    0
 2    1
 3    0
 4    0
 Name: label, dtype: uint8)

In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [25]:
### Vocabulary size 
voc_size = 5000

In [46]:
messages = X.copy()
messages

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object

In [41]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [47]:
### Data cleaning 

ps = PorterStemmer()
clean_Data = []
for i in range(0, len(messages)):
#     print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    clean_Data.append(review)


In [49]:
clean_Data[0:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free']

In [50]:
# convert in one hot encoding 

onehot_rep = [one_hot(word,voc_size) for word in clean_Data]
onehot_rep[1:10]

[[1070, 4710, 2330, 1948, 4357, 1910],
 [3436,
  4393,
  4195,
  4650,
  4778,
  2713,
  3686,
  2550,
  3729,
  2348,
  4131,
  4847,
  2713,
  217,
  4393,
  3658,
  2827,
  2378,
  762,
  4389,
  24],
 [4357, 3449, 4909, 2265, 3047, 4357, 4389, 3632, 4909],
 [2122, 1145, 2639, 1451, 4080, 1253, 4925],
 [3403,
  3396,
  1553,
  671,
  837,
  1870,
  3282,
  1476,
  4355,
  4717,
  1070,
  3059,
  2827,
  2429,
  296,
  1015],
 [2823, 3493, 3282, 3554, 1455, 3282, 2634, 800],
 [55,
  289,
  1877,
  1877,
  676,
  4345,
  2477,
  642,
  3256,
  685,
  1941,
  2943,
  2503,
  2805,
  685],
 [1198,
  4257,
  2607,
  2532,
  1871,
  4589,
  25,
  4406,
  2704,
  869,
  2704,
  733,
  800,
  4204,
  4409],
 [4986,
  1720,
  4357,
  1134,
  4829,
  3041,
  573,
  1289,
  4986,
  1456,
  3436,
  869,
  4986,
  3041,
  4227,
  3436]]

In [52]:
#  Embeding representations

# pad_sequence 
sent_length = 25
embedded_docs = pad_sequences(onehot_rep , padding = 'pre' , maxlen=sent_length )

print(embedded_docs)

[[   0    0    0 ... 4893  303 3265]
 [   0    0    0 ... 1948 4357 1910]
 [   0    0    0 ...  762 4389   24]
 ...
 [   0    0    0 ... 3905  522 4520]
 [   0    0    0 ... 4875   76 3436]
 [   0    0    0 ...  476 4673 1914]]


In [53]:
#  creating model
embeding_vector_feature = 50 
model = Sequential()
model.add(Embedding(voc_size , embeding_vector_feature , input_length=sent_length))
model.add(LSTM(100))    #m  100 is number of nerons
model.add(Dense(1 , activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam' , metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [54]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [56]:
model.fit(X_train , y_train , validation_data=(X_test , y_test) , epochs = 10 ,batch_size=64)

Epoch 1/10
59/59 [==============================] - 8s 55ms/step - loss: 0.3458 - accuracy: 0.8714 - val_loss: 0.1601 - val_accuracy: 0.9723
Epoch 2/10
59/59 [==============================] - 2s 41ms/step - loss: 0.0887 - accuracy: 0.9772 - val_loss: 0.0647 - val_accuracy: 0.9848
Epoch 3/10
59/59 [==============================] - 2s 41ms/step - loss: 0.0346 - accuracy: 0.9898 - val_loss: 0.0493 - val_accuracy: 0.9880
Epoch 4/10
59/59 [==============================] - 3s 45ms/step - loss: 0.0207 - accuracy: 0.9941 - val_loss: 0.0485 - val_accuracy: 0.9859
Epoch 5/10
59/59 [==============================] - 3s 45ms/step - loss: 0.0107 - accuracy: 0.9976 - val_loss: 0.0562 - val_accuracy: 0.9831
Epoch 6/10
59/59 [==============================] - 3s 54ms/step - loss: 0.0077 - accuracy: 0.9981 - val_loss: 0.0517 - val_accuracy: 0.9842
Epoch 7/10
59/59 [==============================] - 3s 46ms/step - loss: 0.0058 - accuracy: 0.9989 - val_loss: 0.0601 - val_accuracy: 0.9848
Epoch 8/10
59

In [57]:
y_pred=model.predict(X_test)
y_pred = y_pred.round()

In [58]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


array([[1591,    2],
       [  18,  228]], dtype=int64)

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9891245241979336

## Create function that Predict the sentance that is reliable or not 

In [64]:
def predict_sentance(str1):
#     clean the sentance
    review = re.sub('[^a-zA-Z]', ' ', str1)
    review = review.lower()
    review = review.split()   
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    # one_hot representation 
    t1 = [one_hot(review , voc_size)]
    # # pad_sequences 
    t1_doc = pad_sequences(t1 , padding = 'pre' , maxlen = 25)
    predict = model.predict(t1_doc).round()
    print(predict)
    if predict == 1.0 :
        print("Spam " , predict)
    else:
        print("Ham " , predict)

In [70]:
t1 = 'please call our customer services center imediatlt '
predict_sentance(t1)

[[1.]]
Spam  [[1.]]


In [72]:
t2 = 'can i call you later '
predict_sentance(t2)

[[0.]]
Ham  [[0.]]
